In [1]:
#!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o
import h2o
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
from h2o.automl import H2OAutoML
from fi_utils import load_dont_overfit
from plt_utils import plot_confusion_matrix
from benchmark_utils import timer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import numpy as np


h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_211"; Java(TM) SE Runtime Environment (build 1.8.0_211-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.211-b12, mixed mode)
  Starting server from /home/bruno/anaconda3/envs/auto-ml/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpq2tyaf4s
  JVM stdout: /tmp/tmpq2tyaf4s/h2o_bruno_started_from_python.out
  JVM stderr: /tmp/tmpq2tyaf4s/h2o_bruno_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,America/Sao_Paulo
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.4
H2O cluster version age:,19 days
H2O cluster name:,H2O_from_python_bruno_af3bd7
H2O cluster total nodes:,1
H2O cluster free memory:,3.455 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [3]:
X_train, X_test, y_train = load_dont_overfit()

X_train['target'] = y_train


In [4]:
train = h2o.H2OFrame.from_python(X_train)
test = h2o.H2OFrame.from_python(X_test)
    

# Identify predictors and response
x = train.columns
y = "target"


# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20,nfolds = 5, seed = 42)
aml.train(x=x, y=y, training_frame=train,)
preds = aml.predict(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [13]:
teste = preds.as_data_frame()

In [16]:

preds_final = [1 if x> 0.5 else 0 for x in teste.as_matrix()]


/home/bruno/anaconda3/envs/auto-ml/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [20]:

submission_autosk = pd.DataFrame({
        "id": X_test["id"],
        "target": preds_final
    })

submission_autosk.to_csv('submission_h2o.csv', index=False)

#print("Accuracy score", sklearn.metrics.accuracy_score(y_test, predictions))